In [1]:
import warnings
warnings.filterwarnings('ignore')

from torch.utils.data import DataLoader, Dataset
import numpy as np
import glob
import collections
from joblib import Parallel, delayed
import collections
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

import librosa, librosa.display
import scipy
from scipy.io import wavfile
import soundfile as sf

import torch
from torch import Tensor
from torchvision import transforms
from torchaudio import transforms as audioTran
from torch.utils.data import Subset
from torch import nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import LambdaLR
from tensorboardX import SummaryWriter

from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

import sys 
import os
#sys.path.append(os.path.abspath(f"D:/deepfake audio"))
import math
import yaml
import copy
from scipy import signal
import wave
import cv2
from sklearn.model_selection import KFold

In [2]:
######### helper functions ############

## For data
def pad(x, max_len = 64600):
    
    x_len = x.shape[0]
    if x_len >= max_len:
        return x[:max_len]
    # need to pad
    num_repeats = int(max_len / x_len)+1
    padded_x = np.tile(x, (1, num_repeats))[:, :max_len][0]
    
    return padded_x

def idx_preprocessing(train_set): 
    idx0 = torch.tensor(train_set.data_sysid) == 0    # get index for each generation method
    idx1 = torch.tensor(train_set.data_sysid) == 1
    idx2 = torch.tensor(train_set.data_sysid) == 2
    idx3 = torch.tensor(train_set.data_sysid) == 3
    idx4 = torch.tensor(train_set.data_sysid) == 4
    idx5 = torch.tensor(train_set.data_sysid) == 5
    idx6 = torch.tensor(train_set.data_sysid) == 6

    train_mask_0 = idx0.nonzero().reshape(-1)
    real_idx = torch.split(train_mask_0, int(len(train_mask_0)/6)) # equally divide real data set  

    train_idx_1 = torch.cat((real_idx[0],idx1.nonzero().reshape(-1)))
    train_idx_2 = torch.cat((real_idx[1],idx2.nonzero().reshape(-1)))
    train_idx_3 = torch.cat((real_idx[2],idx3.nonzero().reshape(-1)))
    train_idx_4 = torch.cat((real_idx[3],idx4.nonzero().reshape(-1)))
    train_idx_5 = torch.cat((real_idx[4],idx5.nonzero().reshape(-1)))
    train_idx_6 = torch.cat((real_idx[5],idx6.nonzero().reshape(-1)))
    
    train_idx = torch.stack((train_idx_1,train_idx_2,train_idx_3,train_idx_4,train_idx_5,train_idx_6))
    
    return train_idx

## For PatchEmbed
def to_2tuple(x):
    if isinstance(x, collections.abc.Iterable):
        return x
    return (x, x)

## For Transformer
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


## For Optimizer
def rate(step, model_size, factor, warmup):

    if step == 0:
        step = 1
    return factor * (
        model_size ** (-0.5) * min(step ** (-0.5), step * warmup ** (-1.5))
    )

def rate_cnn(current_step,  num_training_steps, num_cycles):
    num_warmup_steps = int(0.04 * num_training_steps)
    if current_step < num_warmup_steps:
        return float(current_step) / float(max(1, num_warmup_steps))
    progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
    return 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))

######### helper class ############

class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super(LayerNorm,self).__init__()
        self.gamma = nn.Parameter(torch.ones(features))
        self.beta = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.gamma * (x - mean) / (std + self.eps) + self.beta

In [3]:
###############  Model ####################

In [4]:
class SincConv(nn.Module):   

    @staticmethod
    def to_mel(hz):
        return 2595 * np.log10(1 + hz / 700)

    @staticmethod
    def to_hz(mel):
        return 700 * (10 ** (mel / 2595) - 1)

    def __init__(self, out_channels, kernel_size=128, sample_rate=16000, in_channels=1,
                 stride=1, padding=0, dilation=1, bias=False, groups=1, min_low_hz=50, min_band_hz=50):

        super(SincConv,self).__init__()

        if in_channels != 1:
            msg = "SincConv only support one input channel (here, in_channels = {%i})" % (in_channels)
            raise ValueError(msg)

        self.out_channels = out_channels
        self.kernel_size = kernel_size
        
        if kernel_size%2==0:
            self.kernel_size=self.kernel_size+1
            
        self.stride = stride
        self.padding = padding
        self.dilation = dilation

        if bias:
            raise ValueError('SincConv does not support bias.')
        if groups > 1:
            raise ValueError('SincConv does not support groups.')

        self.sample_rate = sample_rate
        self.min_low_hz = min_low_hz
        self.min_band_hz = min_band_hz

        low_hz = 30
        high_hz = self.sample_rate / 2 - (self.min_low_hz + self.min_band_hz)

        mel = np.linspace(self.to_mel(low_hz),
                          self.to_mel(high_hz),
                          self.out_channels + 1)
        hz = self.to_hz(mel)
        

        self.low_hz_ = nn.Parameter(torch.Tensor(hz[:-1]).view(-1, 1))

        self.band_hz_ = nn.Parameter(torch.Tensor(np.diff(hz)).view(-1, 1))


        n_lin=torch.linspace(0, (self.kernel_size/2)-1, steps=int((self.kernel_size/2))) 
        self.window_=0.54-0.46*torch.cos(2*math.pi*n_lin/self.kernel_size);

        n = (self.kernel_size - 1) / 2.0
        self.band_pass = 2*math.pi*torch.arange(-n, 0).view(1, -1) / self.sample_rate 

 


    def forward(self, x):
        self.band_pass = self.band_pass.to(x.device)

        self.window_ = self.window_.to(x.device)

        low = self.min_low_hz  + torch.abs(self.low_hz_)
        
        high = torch.clamp(low + self.min_band_hz + torch.abs(self.band_hz_),self.min_low_hz,self.sample_rate/2)
        band=(high-low)[:,0]
        
        f_times_t_low = torch.matmul(low, self.band_pass)
        f_times_t_high = torch.matmul(high, self.band_pass)

        band_pass_left=((torch.sin(f_times_t_high)-torch.sin(f_times_t_low))/(self.band_pass/2))*self.window_ # Equivalent of Eq.4 of the reference paper (SPEAKER RECOGNITION FROM RAW WAVEFORM WITH SINCNET). I just have expanded the sinc and simplified the terms. This way I avoid several useless computations. 
        band_pass_center = 2*band.view(-1,1)
        band_pass_right= torch.flip(band_pass_left,dims=[1])
        
        
        band_pass=torch.cat([band_pass_left,band_pass_center,band_pass_right],dim=1)

        
        band_pass = band_pass / (2*band[:,None])
        

        self.filters = (band_pass).view(self.out_channels, 1, self.kernel_size)

        return F.conv1d(x, self.filters, stride=self.stride,
                        padding=self.padding, dilation=self.dilation,
                         bias=None, groups=1) 

In [5]:
class PatchEmbed(nn.Module):
    """ 2D Image to Patch Embedding
    """
    def __init__(self, feature_size, patch_size, embed_dim, in_chans=1):
        super().__init__()
        patch_size = to_2tuple(patch_size)
        self.grid_size = (feature_size[0] // patch_size[0], feature_size[1] // patch_size[1])
        self.num_patches = self.grid_size[0] * self.grid_size[1]

        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        B, H, W = x.shape
        x = x.view(B, 1, H, W)
        x = self.proj(x)
        x = x.flatten(2)
        return x

In [6]:
class EmbedReduce(nn.Module):
    def __init__(self, current_len, seq_size):
        
        super(EmbedReduce,self).__init__()
        self.linear1=nn.Linear(current_len, seq_size[0])
        self.lin_ln1 = nn.LayerNorm(seq_size[0])
        self.linear2=nn.Linear(seq_size[0], seq_size[1])
        self.lin_ln2 = nn.LayerNorm(seq_size[1])
        self.linear3=nn.Linear(seq_size[1], seq_size[2])
        self.lin_ln3 = nn.LayerNorm(seq_size[2])
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.lin_ln1(x)

        x = self.linear2(x)
        x = self.lin_ln2(x)

        x = self.linear3(x)
        x = self.lin_ln3(x)

        return x.transpose(1, 2) 

In [7]:
class FeatureEncoder(nn.Module):
    def __init__(self, encoder_channel, kernel, stride, padding):
        super(FeatureEncoder, self).__init__()
        
        self.device = device
        
        self.mel_transform = audioTran.MelSpectrogram(16000)
       
        self.conv1 = nn.Conv2d(1, encoder_channel[0], kernel, stride=stride, padding=padding)
        nn.init.kaiming_uniform_(self.conv1.weight, a=0.1)
        self.conv1.bias.data.zero_()
        self.encode1 = nn.Sequential(self.conv1, nn.BatchNorm2d(encoder_channel[0]), nn.LeakyReLU(0.2))
        
        
        self.conv2 = nn.Conv2d(encoder_channel[0], encoder_channel[1], kernel,stride=stride, padding=padding)
        nn.init.kaiming_uniform_(self.conv2.weight, a=0.1)
        self.conv2.bias.data.zero_()
        self.encode2 = nn.Sequential(self.conv2,nn.BatchNorm2d(encoder_channel[1]), nn.LeakyReLU(0.2))
        
        
        self.conv3 = nn.Conv2d(encoder_channel[1], encoder_channel[2], kernel,stride=stride, padding=padding)
        nn.init.kaiming_uniform_(self.conv3.weight, a=0.1)
        self.conv3.bias.data.zero_()
        self.encode3 = nn.Sequential(self.conv3, nn.BatchNorm2d(encoder_channel[2]), nn.LeakyReLU(0.2))
        
        self.conv4 = nn.Conv2d(encoder_channel[2], encoder_channel[3], kernel, stride=stride, padding=padding)
        nn.init.kaiming_uniform_(self.conv4.weight, a=0.1)
        self.conv4.bias.data.zero_()
        self.encode4 = nn.Sequential(self.conv4, nn.BatchNorm2d(encoder_channel[3]), nn.LeakyReLU(0.2))
        

        

    def forward(self, x):
        x = self.mel_transform(x)
        
        batch = x.shape[0]
        freq = x.shape[1]
        frame = x.shape[2]
        x = x.view(batch,1,freq,frame)
        
        x = self.encode1(x)
        x = self.encode2(x)
        x = self.encode3(x)
        x = self.encode4(x)
        
        x = x.flatten(2)
        return x.transpose(1, 2) 

In [8]:
## Transformer

In [9]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe) 
        

    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].requires_grad_(False) 
        return self.dropout(x)    

In [10]:
class PositionwiseFeedForward(nn.Module):

    def __init__(self, d_model, d_hidden, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_hidden)
        self.w_2 = nn.Linear(d_hidden, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(self.w_1(x).relu()))
    
    
class SublayerConnection(nn.Module):
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):

        return x + self.dropout(sublayer(self.norm(x)))
    
    
def attention(query, key, value, mask=None, dropout=None):
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = scores.softmax(dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        if mask is not None:
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        query, key, value = [
            lin(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
            for lin, x in zip(self.linears, (query, key, value))
        ]
        x, self.attn = attention(
            query, key, value, mask=mask, dropout=self.dropout
        )
        x = (
            x.transpose(1, 2)
            .contiguous()
            .view(nbatches, -1, self.h * self.d_k)
        )
        del query
        del key
        del value
        return self.linears[-1](x)
    
    
    
class EncoderLayer(nn.Module):
    def __init__(self, d_model, d_hidden, h, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadedAttention(h, d_model)
        self.feed_forward = PositionwiseFeedForward(d_model, d_hidden,)
        self.sublayer = clones(SublayerConnection(d_model, dropout), 2)

    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)
    
    
class Encoder(nn.Module):
    def __init__(self, d_model, d_hidden, N, h, dropout):
        super(Encoder, self).__init__()
        self.layers = clones(EncoderLayer(d_model, d_hidden, h, dropout), N)
        #self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        #return self.norm(x)   
        return x
    
class Transformer(nn.Module):
    def __init__(self, d_model, d_hidden, N, h,dropout=0.1):
        super(Transformer, self).__init__()
        self.d_model = d_model
        self.d_hidden = d_hidden
        
        self.encoder = Encoder(self.d_model, self.d_hidden, N, h, dropout)
        
        print('initialization: xavier')
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
        
    def forward(self, x):
        src_mask = torch.cat((torch.zeros(x.shape[0],1,1),torch.ones(x.shape[0],1,x.shape[1]-1)),dim=2).cuda()
        x = self.encoder(x,src_mask)
        return x

In [11]:
class Net(nn.Module):
    def __init__(self, model_args, device):
        super(Box_v2, self).__init__()
        
        self.device = device

        self.Sinc_conv = SincConv(out_channels = model_args['num_filter'],
                                  kernel_size = model_args['filt_len'],
                                  in_channels = model_args['in_channels'])
        self.feature_dim = int((model_args['samp_len']-model_args['filt_len'])/model_args['max_pool_len']) 
 
        self.lnorm1 = LayerNorm([model_args['num_filter'],self.feature_dim])
        self.leaky_relu = nn.LeakyReLU(0.2)
        
        self.d_embed = model_args['patch_embed']
        self.patchEmbed = PatchEmbed(feature_size = (model_args['num_filter'], self.feature_dim),
                                     patch_size = model_args['patch_size'],
                                     embed_dim = self.d_embed)
        self.seq_len = (model_args['num_filter']//model_args['patch_size'])*(self.feature_dim//model_args['patch_size'])
        self.EmbedReduce = EmbedReduce(current_len = self.seq_len, seq_size = model_args['seq_size'])
        
        self.featureEncoder = FeatureEncoder(model_args['encoder_channel'], model_args['kernel'], 
                                             model_args['stride'], model_args['padding'])
               
        self.posEncode = PositionalEncoding(d_model = self.d_embed, dropout = model_args['drop_out'])
        
        self.transformer = Transformer(d_model = self.d_embed, 
                                       d_hidden = model_args['encoder_hidden'], 
                                       N = model_args['num_block'],
                                       h = model_args['num_head'],
                                       dropout = model_args['drop_out'])
        
        self.gru = nn.GRU(input_size = self.d_embed, hidden_size = model_args['gru_hidden'],
                          num_layers = model_args['gru_layer'], batch_first = True)
        
        self.mlp = nn.Sequential(nn.LayerNorm(model_args['mlp_size'][0]),
                                 nn.Linear(in_features = model_args['mlp_size'][0], out_features = model_args['mlp_size'][1]),
                                 nn.Linear(in_features = model_args['mlp_size'][1], out_features = model_args['nb_classes']))
        
       
    def forward(self, x, y = None,is_test=False):
        batch = x.shape[0]
        len_seq = x.shape[1]
        x_1 = x.view(batch,1,len_seq)

        x_1 = self.Sinc_conv(x_1) 
        x_1 = F.max_pool1d(torch.abs(x_1), 3)
        x_1 = self.lnorm1(x_1)
        x_1 = self.leaky_relu(x_1)
        
        x_1 = self.patchEmbed(x_1)
        x_1 = self.EmbedReduce(x_1)
        
        x_2 = self.featureEncoder(x)
        
        x = torch.cat((x_1, x_2), 1)
        x = self.posEncode(x)
        
        x = self.transformer(x)
        
        self.gru.flatten_parameters()
        x,h_n = self.gru(x)
        
        x = x.transpose(1, 2) 
        
        x = self.mlp(x.mean(dim=1))
        output=F.softmax(x,dim=1)

        return output

In [12]:
######### Loading Dataset #########

In [14]:
###### For ASV DATSET  ########
ASVFile = collections.namedtuple('ASVFile',
    ['speaker_id', 'file_name', 'path', 'sys_id', 'key'])


class ASVDataset(Dataset):
    def __init__(self, data_path=None, label_path=None,transform=None,
                 is_train=True,is_eval=False,feature=None,track=None):
        self.data_path_root = data_path
        self.label_path = label_path
        self.track = track
        self.feature = feature
        self.is_eval = is_eval
        self.transform = transform
        
        if self.is_eval:
            self.sysid_dict = {
            '-': 0,  # bonafide speech
            'A07': 1,
            'A08': 2, 
            'A09': 3, 
            'A10': 4, 
            'A11': 5, 
            'A12': 6,
            'A13': 7, 
            'A14': 8, 
            'A15': 9, 
            'A16': 10, 
            'A17': 11, 
            'A18': 12,
            'A19': 13,    
        }
        else:
            self.sysid_dict = {
            '-': 0,  # bonafide speech         
            'A01': 1, 
            'A02': 2, 
            'A03': 3, 
            'A04': 4, 
            'A05': 5, 
            'A06': 6,        
        }
        
        self.sysid_dict_inv = {v:k for k,v in self.sysid_dict.items()}
        print('sysid_dict_inv',self.sysid_dict_inv)
        
        self.dset_name = 'eval' if is_eval else 'train' if is_train else 'dev'
        print('dset_name',self.dset_name)
        
        self.label_fname = 'eval.trl' if is_eval else 'train.trn' if is_train else 'dev.trl'
        print('label_fname',self.label_fname)
        
        self.label_dir = os.path.join(self.label_path)
        print('protocols_dir',self.label_dir)
        
        track = 'LA' 
        self.prefix = 'ASVspoof2019_{}'.format(track)
        self.audio_files_dir = os.path.join(self.data_path_root, '{}_{}'.format(
            self.prefix, self.dset_name), 'flac')
        print('audio_files_dir',self.audio_files_dir)
        
        self.label_fname = os.path.join(self.label_dir,
            'ASVspoof2019.{}.cm.{}.txt'.format(track, self.label_fname))
        print('label_file',self.label_fname)
        
        
        if (self.dset_name == 'eval'):
            cache_fname = 'cache_ASV_{}.npy'.format(self.dset_name)
            self.cache_fname = os.path.join("/home/menglu/123/Dataset/ASV2019/LA Train/DataCache", cache_fname)
        else:
            cache_fname = 'cache_ASV_{}.npy'.format(self.dset_name)
            self.cache_fname = os.path.join("/home/menglu/123/Dataset/ASV2019/LA Train/DataCache", cache_fname)
            
        if os.path.exists(self.cache_fname):
            self.data_x, self.data_y, self.data_sysid, self.files_meta = torch.load(self.cache_fname)
            print('Dataset loaded from cache', self.cache_fname)
        else: 
            self.files_meta = self.parse_protocols_file(self.label_fname)
            data = list(map(self.read_file, self.files_meta))
            self.data_x, self.data_y, self.data_sysid = map(list, zip(*data))
            if self.transform:
                self.data_x = Parallel(n_jobs=5, prefer='threads')(delayed(self.transform)(x) for x in self.data_x)                          
            torch.save((self.data_x, self.data_y, self.data_sysid, self.files_meta), self.cache_fname)
        
    def __len__(self):
        self.length = len(self.data_x)
        return self.length
   
    def __getitem__(self, idx):
        x = self.data_x[idx]
        y = self.data_y[idx]
        return x, y, self.files_meta[idx]
            
    def read_file(self, meta):   
        #data_x, sample_rate = librosa.load(meta.path,sr=16000)  
        data_x, sample_rate = sf.read(meta.path)
        data_y = meta.key
        return data_x, float(data_y) ,meta.sys_id   

    def parse_line(self, line):
        tokens = line.strip().split(' ')
        if self.is_eval:
            return ASVFile(speaker_id=tokens[0],
                file_name=tokens[1],
                path=os.path.join(self.audio_files_dir, tokens[1] + '.flac'),
                sys_id=self.sysid_dict[tokens[3]],
                key=int(tokens[4] == 'bonafide'))
        return ASVFile(speaker_id=tokens[0],
            file_name=tokens[1],
            path=os.path.join(self.audio_files_dir, tokens[1] + '.flac'),
            sys_id=self.sysid_dict[tokens[3]],
            key=int(tokens[4] == 'bonafide'))

    def parse_protocols_file(self, label_fname):
        lines = open(label_fname).readlines()
        files_meta = map(self.parse_line, lines)
        return list(files_meta)

In [16]:
# Dataloader TRAINING
database_path = "/home/menglu/123/Dataset/ASV2019/LA Train"
label_path = "/home/menglu/123/Dataset/ASV2019/LA Train/ASVspoof2019_LA_cm_protocols"
transform = transforms.Compose([
    lambda x: pad(x),
    lambda x: Tensor(x)
])

train_set = ASVDataset(data_path=database_path,label_path=label_path,is_train=True,transform=transform)
train_idx = idx_preprocessing(train_set)

# Dataloader  VALIDATION
dev_set = ASVDataset(data_path = database_path,label_path = label_path,is_train=False, transform=transform)
dev_loader = DataLoader(dev_set, batch_size=32, shuffle=True)

sysid_dict_inv {0: '-', 1: 'A01', 2: 'A02', 3: 'A03', 4: 'A04', 5: 'A05', 6: 'A06'}
dset_name train
label_fname train.trn
protocols_dir /home/menglu/123/Dataset/ASV2019/LA Train/ASVspoof2019_LA_cm_protocols
audio_files_dir /home/menglu/123/Dataset/ASV2019/LA Train/ASVspoof2019_LA_train/flac
label_file /home/menglu/123/Dataset/ASV2019/LA Train/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt
Dataset loaded from cache /home/menglu/123/Dataset/ASV2019/LA Train/DataCache/cache_ASV_train.npy
sysid_dict_inv {0: '-', 1: 'A01', 2: 'A02', 3: 'A03', 4: 'A04', 5: 'A05', 6: 'A06'}
dset_name dev
label_fname dev.trl
protocols_dir /home/menglu/123/Dataset/ASV2019/LA Train/ASVspoof2019_LA_cm_protocols
audio_files_dir /home/menglu/123/Dataset/ASV2019/LA Train/ASVspoof2019_LA_dev/flac
label_file /home/menglu/123/Dataset/ASV2019/LA Train/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.dev.trl.txt
Dataset loaded from cache /home/menglu/123/Dataset/ASV2019/LA Train/DataCache/cache_ASV_dev.npy


In [ ]:
######### Start training #########

In [17]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=.25, gamma=2):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha, (float, int)): self.alpha = torch.Tensor([alpha, 1 - alpha]).cuda()
        if isinstance(alpha, list): self.alpha = torch.Tensor(alpha).cuda()

    def forward(self, inputs, targets):
        BCE_loss = F.cross_entropy(inputs, targets, reduce=False)
        at = self.alpha.gather(0, targets.data.view(-1))
        pt = torch.exp(-BCE_loss)
        F_loss = at*(1-pt)**self.gamma * BCE_loss
        return torch.mean(F_loss)

    
def evaluate_accuracy(data_loader, model, device):
    num_correct = 0.0
    num_total = 0.0
    model.eval()
    for batch_x, batch_y, batch_meta in tqdm(data_loader):
    #for batch_x, batch_y in data_loader:
        batch_size = batch_x.size(0)
        num_total += batch_size
        batch_x = batch_x.to(device)
        batch_y = batch_y.view(-1).type(torch.int64).to(device)
        batch_out = model(batch_x,batch_y)
        _, batch_pred = batch_out.max(dim=1)
        num_correct += (batch_pred == batch_y).sum(dim=0).item()
    return 100 * (num_correct / num_total)



def train_epoch_with_swap(train_A_loader, train_B_loader, idx, model_A, model_B, lr, 
                          optim_A, optim_B, device, buffer_vec, scheduler_A = None, scheduler_B = None):
    
    running_loss = 0
    num_correct = 0.0
    model_A.train()
    
    weight = torch.FloatTensor([1.0, 9.0]).to(device)
    CEloss = nn.CrossEntropyLoss(weight=weight)
    
    for train_A_set, train_B_set in tqdm(zip(train_A_loader,train_B_loader), total = len(train_A_loader)):
        
        A_x = train_A_set[0]
        A_y = train_A_set[1]
        batch_size = A_x.size(0)
        idx += 1
        
        A_x = A_x.to(device)
        A_y = A_y.view(-1).type(torch.int64).to(device)
        A_out = model_A(A_x,A_y)
        
        CE_Loss_from_A = CEloss(A_out, A_y)
        Focal_Loss_from_A = FocalLoss()(A_out, A_y)

        batch_loss_from_A = CE_Loss_from_A + Focal_Loss_from_A
        _, batch_pred_from_A = A_out.max(dim=1)

        batch_acc = ((batch_pred_from_A == A_y).sum(dim=0).item() / batch_size)*100
        batch_loss = batch_loss_from_A.item() 
  
        
        optim_A.zero_grad()
        batch_loss_from_A.backward()
        optim_A.step()
        if scheduler_A !=None:
            scheduler_A.step()
        
        writer.add_scalar('train_accuracy', batch_acc, idx)
        writer.add_scalar('loss', batch_loss, idx)
                
          
        model_A_dict = model_A.state_dict()
        model_B_dict = model_B.state_dict()
        buffer_dict = {k: v for k, v in model_A_dict.items() if k not in buffer_vec}
        model_B_dict.update(buffer_dict)
        model_B.load_state_dict(model_B_dict)
        model_B.train()
        
        B_x = train_B_set[0]
        B_y = train_B_set[1]   
        B_x = B_x.to(device)
        B_y = B_y.view(-1).type(torch.int64).to(device)
        B_out = model_B(B_x,B_y)
        
        CE_Loss_from_B = CEloss(B_out, B_y)
        Focal_Loss_from_B = FocalLoss()(B_out, B_y)

        batch_loss_from_B = CE_Loss_from_B + Focal_Loss_from_B
        _, batch_pred_from_B = B_out.max(dim=1)
        
        optim_B.zero_grad()
        batch_loss_from_B.backward()
        optim_B.step()
        if scheduler_B !=None:
            scheduler_B.step() 
    return idx

In [19]:
## start experiment
kfold = KFold(n_splits = config['data']['k_fold'], shuffle=True)
train_fold_list = list(kfold.split(list(range(config['data']['num_method']))))

np.random.seed(1234)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# GPU device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Parameter
config = yaml.safe_load(open('try_optim.yaml'))
batch_size = config['batch_size']
lr = config['lr']
warmup = config['warmup']        
num_epochs = config['epoch']
d_model = config['model']['patch_embed']
num_filter = config['model']['num_filter']
num_block = config['model']['num_block']
num_head = config['model']['num_head']

# Model Initialization
model_A = Net(config['model'],device).to(device)
select_param = [k for k, v in model_A.named_parameters() if k.startswith("FeatureEncoder")]
for name, param in model_A.named_parameters(): 
    if name in select_param:
        param.requires_grad = False
        
optim_A = torch.optim.Adam(filter(lambda param: param.requires_grad, model_A.parameters()), 
                           lr = lr, betas=(0.9, 0.98), eps=1e-9)
lr_scheduler_A = LambdaLR(optimizer=optim_A,
                        lr_lambda=lambda step: rate(step, d_model, factor=1, warmup=config["warmup"]),)
select_buffer_vec = [k for k, v in model_A.state_dict().items() if k.startswith("FeatureEncoder")]



model_B = Net(config['model'],device).to(device)
select_param = [k for k, v in model_B.named_parameters() if k.startswith("FeatureEncoder")]
for name, param in model_B.named_parameters(): 
    if name not in select_param:
        param.requires_grad = False

optim_B = torch.optim.Adam(filter(lambda param: param.requires_grad, model_B.parameters()),
                             lr = lr, betas=(0.9, 0.98), eps=1e-9)
lr_scheduler_B = LambdaLR(optimizer=optim_B,
                        lr_lambda=lambda step: rate_cnn(step, num_training_steps=55000, num_cycles=0.5),)



# model log
model_tag = '{}_{}_{}_{}_{}_{}'.format(batch_size,d_model, num_filter, num_block, num_head,lr)
model_save_path = os.path.join('/home/menglu/123/Deepfake/ASV_v1', model_tag)

if os.path.exists(model_save_path)==False:
    os.makedirs(model_save_path)
print(model_tag)
writer = SummaryWriter('logs/{}'.format(model_tag))

# init
best_valid_acc = 40
index = 0

for epoch in range(num_epochs):
    train_fold_idx = epoch % config['data']['k_fold']
    
    for i in range(1,len(train_fold_list[train_fold_idx][0])):  # get index from train_A set
        if i == 1:
            idx = train_fold_list[train_fold_idx][0][i]
            train_A_indices = torch.concat((train_idx[0],train_idx[1]))
        else:
            idx = train_fold_list[train_fold_idx][0][i]
            train_A_indices = torch.concat((train_A_indices,train_idx[idx]))
            
    train_B_set_idx1 = train_fold_list[train_fold_idx][1][0]
    train_B_set_idx2 = train_fold_list[train_fold_idx][1][1]
    train_B_indices = torch.concat((train_idx[train_B_set_idx1],train_idx[train_B_set_idx2]))
    
    train_A_set = Subset(train_set, train_A_indices)
    train_B_set = Subset(train_set, train_B_indices)
    
    # get two sets of training data
    train_A_loader = DataLoader(train_A_set, batch_size=batch_size, shuffle=True, drop_last=True)
    train_B_loader = DataLoader(train_B_set, batch_size=batch_size, shuffle=True, drop_last=True)
    
    index = train_epoch_with_swap(train_A_loader, train_B_loader, index,model_A, model_B, lr, 
                                             optim_A, optim_B, device, select_buffer_vec,
                                             scheduler_A = lr_scheduler_A, scheduler_B = lr_scheduler_B)
    
    if epoch <= num_epochs-1:
        model_A_dict = model_A.state_dict()
        model_B_dict = model_B.state_dict()
        buffer_dict = {k: v for k, v in model_B_dict.items() if k in select_buffer_vec}
        model_A_dict.update(buffer_dict)
        model_A.load_state_dict(model_A_dict)
        
    valid_acc = evaluate_accuracy(dev_loader, model_A, device)
    writer.add_scalar('valid_accuracy', valid_acc, epoch)
    print('\n{} - {:.4f} - {} - {}'.format(epoch,valid_acc, optim_A.param_groups[0]['lr'], optim_B.param_groups[0]['lr']))
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        torch.save(model_A.state_dict(), os.path.join(model_save_path, 'epoch_{}.pth'.format(epoch)))

writer.close()    

200
initialization: xavier
initialization: xavier
29658194
v5_32_512_128_6_8_0.001


100%|█████████████████████████████████████████| 777/777 [01:00<00:00, 12.90it/s]



0 - 78.9285 - 1.1667261889578034e-08 - 0.00012


100%|█████████████████████████████████████████| 777/777 [00:58<00:00, 13.36it/s]



1 - 69.6385 - 2.3334523779156068e-08 - 0.00024


...


197 - 97.8667 - 1.9329937995448063e-07 - 6.572141796596376e-06


100%|█████████████████████████████████████████| 777/777 [00:56<00:00, 13.78it/s]



198 - 97.9794 - 1.928130914357838e-07 - 5.363833518505834e-06


100%|█████████████████████████████████████████| 777/777 [00:56<00:00, 13.73it/s]


199 - 97.7258 - 1.9233045465520742e-07 - 4.277569313094809e-06
